In [1]:
from datasets.datamodule import DEEPscreenDataModule
from engine.system import DEEPScreenClassifier
from lightning import Trainer
# from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks import RichProgressBar
import pandas as pd

In [2]:
data = pd.read_csv("../../.data/processed/CHEMBL5567.csv")

In [3]:
hyperparameters = {"fully_layer_1":256,"fully_layer_2":32,"drop_rate":0.5,"learning_rate":0.0001,"batch_size":32}
checkpoint_filename = f"deepscreen-{{epoch:02d}}-{{val_loss:.2f}}-{{val_mcc:.2f}}-{hyperparameters['fully_layer_1']}-{hyperparameters['fully_layer_2']}-{hyperparameters['learning_rate']}-{hyperparameters['drop_rate']}-{hyperparameters['batch_size']}"
checkpoint_callback = ModelCheckpoint(dirpath="../../.experiments/chembl5567", save_top_k=3, monitor="val_mcc",filename=checkpoint_filename)
early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=60)
trainer = Trainer(callbacks=[checkpoint_callback,early_stopping,RichProgressBar(leave=True)],max_epochs=200,enable_model_summary=False,log_every_n_steps=38)
model = DEEPScreenClassifier(**hyperparameters)
datamodule = DEEPscreenDataModule(data=data,target_id="chembl5567",batch_size=32,experiment_result_path="../../.experiments/chembl5567",data_split_mode="non_random_split",tmp_imgs=True)
trainer.test(model,datamodule=datamodule,ckpt_path="/home/sjinich/disco/TrypanoDEEPscreen/.experiments/chembl5567/deepscreen-epoch=08-val_loss=0.60-val_mcc=-0.03-256-32-0.0001-0.5-32.ckpt")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/big/lab/sjinich/venv_che/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
INFO: Training this hyperparameters [('fully_layer_1', 256), ('fully_layer_2', 32), ('drop_rate', 0.5), ('learning_rate', 0.0001), ('batch_size', 32)]
INFO: Using a total of 1876 datapoints
INFO: non_random_split datasets splited train=1200,validation=300,test=376
Restoring states from the checkpoint path at /home

Output()

/big/lab/sjinich/venv_che/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7420212626457214     │
│          test_f1          │    0.29197078943252563    │
│         test_loss         │    0.5912715792655945     │
│         test_mcc          │    0.14264878630638123    │
│         test_pres         │           0.25            │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5912715792655945,
  'test_acc': 0.7420212626457214,
  'test_f1': 0.29197078943252563,
  'test_mcc': 0.14264878630638123,
  'test_pres': 0.25}]